In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

import numpy as np
from spreg import OLS
import statsmodels.api as sm

In [2]:
# Read in all the data

AQMS_df=pd.read_csv('data/AQMS_readings.csv')
Rd_gdf=gpd.read_file('data/london_Road.shp')
Gsp_gdf=gpd.read_file('data/LD_GreenSpace.shp')
loc_gdf=gpd.read_file('data/AQMS_loc.shp')
cond=pd.read_csv('data/cond.csv')

In [3]:
AQMS_df.head()

,DateTime,Value,Site
0,2019-01-01,7.458333,CT2
1,2019-01-02,14.913043,CT2
2,2019-01-03,24.083333,CT2
3,2019-01-04,10.541667,CT2
4,2019-01-05,18.125000,CT2


In [5]:
Rd_gdf.head()

,fictitious,identifier,class,roadNumber,name1,name1_lang,name2,name2_lang,formOfWay,length,primary,trunkRoad,loop,startNode,endNode,structure,nameTOID,numberTOID,function,geometry
0,false,8CC0934A-4A4A-435A-BEBB-521AD3E8C143,Not Classified,None,The Bridlepath,None,None,None,Single Carriageway,302,false,false,false,C19EB0EA-BED0-42A6-B4B0-EBDEFA73B800,EE0E8646-C1A3-4A78-94FF-6667EC2D5EAC,None,osgb5000005123758573,None,Restricted Local Access Road,"LINESTRING Z (531347.102 155899.478 0.000, 531..."
1,false,ECE86DA8-118A-46AB-8D5D-56F68B96E7BB,Unclassified,None,Ditches Lane,None,None,None,Single Carriageway,1599,false,false,false,5818762C-F0E5-4D3C-B28A-43D3BB81347C,A704D38C-5674-4270-A4C2-D936F9CE8429,None,osgb4000000030481053,None,Minor Road,MULTILINESTRING Z ((530900.469 155851.102 0.00...
2,false,960A1B1E-15CD-4E9C-816C-4F79CB0442E7,A Road,A233,Main Road,None,None,None,Single Carriageway,12,false,false,false,0B3C9057-F8D0-469F-BAAD-EF66693A6756,52F19A51-09CA-4BE2-932D-A6F5E0827706,None,osgb4000000030481070,osgb4000000030575917,A Road,"LINESTRING Z (543449.179 156812.005 0.000, 543..."
3,false,0E0182BB-7E46-4250-B9EE-37D58BA0E73C,Unclassified,None,Grays Road,None,None,None,Single Carriageway,16,false,false,false,52F19A51-09CA-4BE2-932D-A6F5E0827706,03D54909-5CE1-44C7-B9FE-E55812121CE9,None,osgb4000000030485658,None,Minor Road,"LINESTRING Z (543446.000 156819.000 0.000, 543..."
4,false,A6456BD8-2D7F-4CE9-9192-112965FA7AD1,Unclassified,None,Old Fox Close,None,None,None,Single Carriageway,230,false,false,false,FCE37925-FBDA-4361-A036-32922B6B3752,CDC3FDC8-BE9E-426C-8FAD-B2133897E040,None,osgb4000000030514396,None,Local Road,"LINESTRING Z (531860.000 156862.000 0.000, 531..."


In [6]:
Gsp_gdf.head()

,function,distName1,distName2,distName3,distName4,geometry
0,Religious Grounds,St Thomas's Church,None,None,None,"POLYGON Z ((554094.150 193959.630 0.000, 55406..."
1,Other Sports Facility,None,None,None,None,"POLYGON Z ((554242.200 193852.330 0.000, 55419..."
2,Public Park Or Garden,The Green,None,None,None,"MULTIPOLYGON Z (((538620.740 194410.900 0.000,..."
3,Golf Course,Chingford Golf Course,None,None,None,"MULTIPOLYGON Z (((539444.210 194937.620 0.000,..."
4,Allotments Or Community Growing Spaces,None,None,None,None,"POLYGON Z ((538849.200 193723.300 0.000, 53881..."


In [7]:
loc_gdf.head()

,siteid,sitename,geometry
0,BX9,Bexley - Slade Green FDMS,POINT (551862.205 176375.976)
1,BL0,Camden - Bloomsbury,POINT (530120.048 182038.807)
2,CD9,Camden - Euston Road,POINT (529900.870 182666.124)
3,CD1,Camden - Swiss Cottage,POINT (526629.730 184391.024)
4,CT2,City of London - Farringdon Street,POINT (531622.273 181213.818)


In [4]:
cond.head()

,DateTime,BP,Tmp,RH
0,2019-01-01,1029.864583,8.729167,75.312500
1,2019-01-02,982.156250,1.885417,96.145833
2,2019-01-03,1010.770833,9.300000,85.805556
3,2019-01-04,1011.833333,8.816667,73.750000
4,2019-01-05,1010.125000,12.500000,72.222222


In [8]:
test_df=AQMS_df[AQMS_df['Site']=='CT2'].merge(cond).dropna()

test_df.corr()

,Value,BP,Tmp,RH
Value,1.000000,0.184627,-0.095147,0.041614
BP,0.184627,1.000000,0.057986,-0.379641
Tmp,-0.095147,0.057986,1.000000,-0.452950
RH,0.041614,-0.379641,-0.452950,1.000000


In [20]:
AQMS_df.merge(cond,how='left',on='DateTime').corr()

,Value,BP,Tmp,RH
Value,1.000000,0.247502,-0.150348,-0.037343
BP,0.247502,1.000000,0.055893,-0.378787
Tmp,-0.150348,0.055893,1.000000,-0.451795
RH,-0.037343,-0.378787,-0.451795,1.000000


In [11]:
test_df.dtypes

DateTime     object
Value       float64
Site         object
BP          float64
Tmp         float64
RH          float64
dtype: object

In [18]:
m_test=sm.OLS(test_df['Value'].values,test_df['BP'].values)
m_test_fit=m_test.fit()
print(m_test_fit.summary())

                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.761
Model:                            OLS   Adj. R-squared (uncentered):              0.760
Method:                 Least Squares   F-statistic:                              1130.
Date:                Wed, 02 Feb 2022   Prob (F-statistic):                   1.58e-112
Time:                        10:45:15   Log-Likelihood:                         -1242.5
No. Observations:                 357   AIC:                                      2487.
Df Residuals:                     356   BIC:                                      2491.
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

In [ ]:
# reindex loc_gdf and set buffer zones around each site (1km)

loc_gdf=loc_gdf.set_index('siteid')
loc_gdf['buffer_1km']=loc_gdf['geometry'].buffer(1000)
loc_gdf.head()

In [ ]:
# Get all green spaces

Gsp=Gsp_gdf['geometry'].unary_union

In [ ]:
# Set buffer zones for every road (50m)

Rd_gdf['buffer_50m']=Rd_gdf['geometry'].buffer(50)

In [ ]:
Rd=Rd_gdf['geometry'].unary_union

In [ ]:
# Get all road buffer zones

Rd_buffer=Rd_gdf['buffer_50m'].unary_union

In [ ]:
# Get all near-road green spaces

nRd_Gsp=Gsp.intersection(Rd_buffer)

In [ ]:
AQMS_shp=[]
for site in loc_gdf.index:
    AQMS_shp.append(loc_gdf.loc[site,'buffer_1km'])

In [ ]:
Rd_shp=[]
Gsp_shp=[]
nRd_Gsp_shp=[]
for poly in AQMS_shp:
    Rd_shp.append(poly.intersection(Rd))
    Gsp_shp.append(poly.intersection(Gsp))
    nRd_Gsp_shp.append(poly.intersection(nRd_Gsp))
print(len(Rd_shp),len(Gsp_shp),len(nRd_Gsp_shp))

In [ ]:
loc_gdf['Road']=Rd_shp
loc_gdf['GreenSpace']=Gsp_shp
loc_gdf['NR_GreenSpace']=nRd_Gsp_shp
loc_gdf=loc_gdf.set_geometry('Road').set_geometry('GreenSpace').set_geometry('NR_GreenSpace')
loc_gdf

In [ ]:
loc_gdf.dtypes

In [ ]:
london=gpd.read_file('data/london_boundary.shp')

In [ ]:
fig,ax=plt.subplots(1,figsize=(15,13))

london.plot(color='lightgrey',ax=ax)
loc_gdf['buffer_1km'].plot(color='silver',ax=ax)
loc_gdf['GreenSpace'].plot(label='Green space',color='limegreen',ax=ax)
loc_gdf['NR_GreenSpace'].plot(label='Near_road green space',color='darkgreen',ax=ax)
loc_gdf['Road'].plot(color='black',label='Road',linewidth=0.1,ax=ax)
loc_gdf['geometry'].plot(markersize=5,marker='^',color='blue',label='Air quality monitoring site',ax=ax)

ax.axis('off')

legend=ax.legend(loc='best',shadow=True,fontsize=15)

#plt.savefig('sample1.png',facecolor='black',dpi=500)
plt.show()

In [ ]:
loc_gdf['min_dis']=pd.Series(dtype='float64')
for index, row in loc_gdf.iterrows():
    dis=[]
    for i, v in loc_gdf['geometry'].iteritems():
        dis.append(row['geometry'].distance(v))
    dis.remove(0)
    loc_gdf.loc[index,'min_dis']=min(dis)

In [ ]:
loc_gdf[loc_gdf['min_dis']<=1500]

In [ ]:
loc_gdf.drop(['BL0','GR4'],inplace=True)

In [ ]:
df=AQMS_df.drop(AQMS_df[AQMS_df['Site'].isin(['BL0','GR4'])].index).reset_index(drop=True)
df

In [ ]:
df=df.merge(loc_gdf,left_on='Site',right_index=True)
df

In [ ]:
fig,ax=plt.subplots(1,figsize=(15,13))

london.plot(color='lightgrey',ax=ax)
loc_gdf['buffer_1km'].plot(color='silver',ax=ax)
loc_gdf['GreenSpace'].plot(label='Green space',color='limegreen',ax=ax)
loc_gdf['NR_GreenSpace'].plot(label='Near_road green space',color='darkgreen',ax=ax)
loc_gdf['Road'].plot(color='black',label='Road',linewidth=0.1,ax=ax)
loc_gdf['geometry'].plot(markersize=5,marker='^',color='blue',label='Air quality monitoring site',ax=ax)

ax.axis('off')

legend=ax.legend(loc='best',shadow=True,fontsize=15)

#plt.savefig('sample1.png',facecolor='black',dpi=500)
plt.show()

In [ ]:
loc_gdf['buffer_area']=loc_gdf['buffer_1km'].area

In [ ]:
loc_gdf['pct_Gsp_area']=loc_gdf['GreenSpace'].area/loc_gdf['buffer_area'] # percentage of gsp area
loc_gdf['pct_nR_Gsp_area']=loc_gdf['NR_GreenSpace'].area/loc_gdf['buffer_area'] # percentage of near road gsp area

In [ ]:
loc_gdf.head()

In [ ]:
df=df.merge(cond,on='ReadingDateTime',how='left')

In [ ]:
df

In [ ]:
df.drop('min_dis',axis=1,inplace=True)

In [ ]:
var=['pct_Gsp_area','pct_nR_Gsp_area','BP','Tmp','RH']

In [ ]:
from spreg import OLS

m_multi=OLS(y=df.dropna()['Value'].values, name_y='PM2.5 reading',
            x=df.dropna()[var].values,name_x=var)
print(m_multi.summary)